# Create Cloud Provider Configuration
### Setup

First, import the required packages.

Next, specify some important variables:
* `platform_address`: The address of the Yugabyte Platform API
* `platform_api_key`: The API key used to authenticate with the Platform API

Finally, open a HTTP connection to the Yugabyte Platform API.

In [21]:
import configparser
import os
import http.client
import json
from pprint import pprint

platform_address = os.getenv('API_BASE_URL', "portal.dev.yugabyte.com")
platform_api_key = os.getenv('YB_API_KEY')

conn = http.client.HTTPConnection(f"{platform_address}")

headers = {
  'Content-Type': "application/json",
  'X-AUTH-YW-API-TOKEN': f"{platform_api_key}"
}

### Get Session Info

Make an API call to session endpoint to determine customer UUID.

In [22]:
conn.request("GET", "/api/v1/session_info", headers=headers)

res = conn.getresponse()
data = res.read()
session_info = json.loads(data)
customer_uuid = session_info['customerUUID']

print('Customer UUID:\n%s' % customer_uuid)

Customer UUID:
11d78d93-1381-4d1d-8393-ba76f47ba7a6


## GCP Provider
### Define GCP Provider

In this example, we will create a GCP provider. First, load in GCP service account credentials JSON file. Then, define a `Provider` object with the credentials and desired configuration.

In [ ]:
gcp_creds_filename = os.getenv('GCE_CREDENTIALS_FILE_PATH')
gcp_creds_file = open(gcp_creds_filename, 'r')
gcp_creds = json.loads(gcp_creds_file.read())

new_provider = {
    "code": "gcp",
    "config": {
        **gcp_creds,
        "YB_FIREWALL_TAGS": "cluster-server",
    },
    "destVpcId": "yugabyte-network",
    "name": "my-gcp-provider",
    "regions": [
        {
            "code": "us-central1",
            "name": "us-central1",
            "zones": [],
        },
    ],
    "sshPort": 54422,
    "airGapInstall": False,
}

### Create Provider

Make API call to provider endpoint to create new provider.

In [ ]:
url = f"/api/v1/customers/{customer_uuid}/providers"
conn.request("POST", url, json.dumps(new_provider), headers)
res = conn.getresponse()
print(json.dumps(json.load(res), indent=2))

## AWS Provider
### Define AWS Provider

In this example, we will create an AWS provider. You need to have set `AWS_ACCESS_KEY_ID` and `AWS_SECRET_ACCESS_KEY` environment variables.

In [29]:
config = configparser.RawConfigParser()
config.read(os.path.expanduser('~/.aws/credentials'))
# print (config.sections())
new_provider = {
  "code": "aws",
  "name": "my-aws-provider",
  "config": {
    "AWS_ACCESS_KEY_ID": f"{config.get('default', 'aws_access_key_id')}",
    "AWS_SECRET_ACCESS_KEY": f"{config.get('default', 'aws_secret_access_key')}"
  },
  "regions": [
      {
          "code": "us-west-2",
          "name": "us-west-2",
          "zones": [],
      },
  ]
}
#pprint(new_provider)

### Create Provider

Make API call to provider endpoint to create new provider.

In [24]:
url = f"/api/v1/customers/{customer_uuid}/providers"
conn.request("POST", url, json.dumps(new_provider), headers)
res = conn.getresponse()
pprint(json.load(res))

{'resourceUUID': '3ef96e2b-4940-407a-ae55-e13971285791',
 'taskUUID': '0fe7fed6-8b3b-4c60-8ad2-124b217ad83a'}
